Importar Módulos

In [ ]:
import pandas as pd
import pathlib
import win32com.client as win32

Ler os arquivos excel e realizar tratamento de dados

In [ ]:
produtos_df = pd.read_csv('Contoso - Cadastro Produtos.csv', sep=';', decimal=',')
emails_df = pd.read_excel('Contoso - Contatos.xlsx')
vendas_df = pd.read_csv('Contoso - Vendas - 2017.csv', sep=';')
lojas_df = pd.read_csv('Contoso - Lojas.csv', sep=';')

vendas_df = vendas_df.merge(lojas_df, on='ID Loja')
vendas_df = vendas_df.merge(produtos_df, on='ID Produto')
vendas_df = vendas_df[['Data da Venda', 'ID Loja', 'ID Produto', 'Quantidade Vendida', 'Nome da Loja', 'Preco Unitario', 'Numero da Venda']]
vendas_df['Preco Unitario'] = vendas_df[['Preco Unitario']].multiply(vendas_df['Quantidade Vendida'], axis='index')
vendas_df.rename(columns={'Preco Unitario': 'Valor Total'}, inplace=True)
vendas_df['Data da Venda'] = pd.to_datetime(vendas_df['Data da Venda'], format='%d/%m/%Y')
display(vendas_df)

Coletar o último dia (Dia do Indicador)

In [ ]:
dia_indicador = vendas_df['Data da Venda'].max()
print(dia_indicador)
print('{}/{}'.format(dia_indicador.day, dia_indicador.month))

Construir um dicionário que contém um Data Frame para cada loja

In [ ]:
dicionario_lojas = {}
for loja in lojas_df['Nome da Loja']:
    dicionario_lojas[loja.strip()] = vendas_df.loc[vendas_df['Nome da Loja'] == loja, :]

display(dicionario_lojas['Loja Contoso Fall City'])

Enviar os arquivos de cada loja para cada pasta de Backup

In [ ]:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
arquivos_pasta_backup = caminho_backup.iterdir()

lista_nomes_backup = [arquivo.name.strip() for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    
    dicionario_lojas[loja].to_excel(local_arquivo)

Definindo todas as metas

In [ ]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

Enviando os e-mails para cada loja com seu respectivo OnePage

In [ ]:
for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data da Venda'] == dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Total'].sum()
    faturamento_dia = vendas_loja_dia['Valor Total'].sum()

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['ID Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['ID Produto'].unique())

    #ticket médio
    valor_venda = vendas_loja.groupby('Numero da Venda').sum(numeric_only=True)
    ticket_medio_ano = valor_venda['Valor Total'].mean()
    valor_venda_dia = vendas_loja_dia.groupby('Numero da Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Total'].mean()
    
    #Enviando e-mails
    outlook = win32.Dispatch('outlook.application')

    nome = emails_df.loc(emails_df['Nome da Loja'] == loja, 'Nome do Vendedor').values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails_df.loc(emails_df['Nome da Loja'] == loja, 'Email da Loja').values[0]
    mail.Subject = f'One Page Dia {dia_indicador.day}/{dia_indicador.month} - {loja}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
        
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
        
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
        
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'

    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'

    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem ({dia_indicador.day}/{dia_indicador.month}) da {loja} foi: </p>

    <table>
        <tr>
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R$ {faturamento_dia:.2f}</td>
            <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
            <td style="text-align: center">◙</td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{qtde_produtos_dia}</td>
            <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
            <td style="text-align: center">◙</td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
            <td style="text-align: center">R$ {meta_ticketmedio_dia:.2f}</td>
            <td style="text-align: center">◙</td>
        </tr>
    </table>
    <br>
    <table>
        <tr>
            <th>Indicador</th>
            <th>Valor ano</th>
            <th>Meta ano</th>
            <th>Cenário ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R$ {faturamento_ano:.2f}</td>
            <td style="text-align: center">R$ {meta_faturamento_ano:.2f}</td>
            <td style="text-align: center">◙</td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{qtde_produtos_ano}</td>
            <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
            <td style="text-align: center">◙</td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {ticket_medio_ano:.2f}</td>
            <td style="text-align: center">R$ {meta_ticketmedio_ano:.2f}</td>
            <td style="text-align: center">◙</td>
        </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Igor</p>
    '''

    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()

Criando os arquivos de visão geral para serem enviados para a Diretoria 

In [ ]:
faturamento_lojas = vendas_df.groupby('Nome da Loja')[['Nome da Loja', 'Valor Total']].sum(numeric_only=True)
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Total', ascending=False)
display(faturamento_lojas_ano)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas_df.loc[vendas_df['Data da Venda'] == dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Nome da Loja')[['Nome da Loja', 'Valor Total']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Total', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

Enviando e-mail de visão geral para a Diretoria

In [ ]:
#Enviar e-mail para Diretoria
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails_df.loc(emails_df['Nome da Loja'] == 'Diretoria', 'Email da Loja').values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R$ {faturamento_lojas_dia.iloc[0, 0]:.2f}.
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R$ {faturamento_lojas_dia.iloc[-1, 0]:.2f}.

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R$ {faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R$ {faturamento_lojas_dia.iloc[-1, 0]:.2f}.

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Igor
'''

attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()